In [ ]:
from google.cloud import bigquery
import pandas as pd
import datetime
from datetime import timedelta
import time
from utility_functions import create_dataset

In [ ]:
project_id="" #Put your project id

In [ ]:
client = bigquery.Client(project=f'{project_id}')

In [ ]:
create_dataset(project=f'{project_id}', dataset_id='BIGQUERY_STATS', confirm_dataset='n', location='australia-southeast1')

In [ ]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [ ]:
!bq ls -n 1000 -d --project_id=project_id --format=prettyjson | jq '.[] | select(.location == "australia-southeast1") .datasetReference .datasetId' > australianDatasets.txt 

In [ ]:
!bq ls -d --project_id=project_id --format=prettyjson | jq '.[] | select(.location == "EU") .datasetReference .datasetId' > EuDatasets.txt 

In [ ]:
eu_datasets = pd.read_csv('EuDatasets.txt', header=None,names=['Datasets'])

In [ ]:
au_datasets = pd.read_csv('australianDatasets.txt', header=None, names=['Datasets'])

In [ ]:
au_datasets

In [ ]:
eu_datasets = [dataset for dataset in eu_datasets['Datasets']]
au_datasets = [dataset for dataset in au_datasets['Datasets']]

In [ ]:
eu_datasets = chunks(eu_datasets, 5)
au_datasets = chunks(au_datasets, 50)

In [ ]:
df_au = []
for ds in au_datasets:
    query = ""
    for i in ds:            
        q = "SELECT * FROM `{}.__TABLES__` UNION ALL ".format(f'{project_id}.'+i)
        query = query + q
    query = query[:-10]
    try:
        dataset = pd.read_gbq(query, project_id=project_id, dialect = 'standard')
        df_au.append(dataset)
    except Exception as e:
        print(e)
df_au = pd.concat(df_au)

In [ ]:
df_eu = []
for ds in eu_datasets:
    query = ""
    for i in ds:            
        q = "SELECT * FROM `{}.__TABLES__` UNION ALL ".format(f'{project_id}.'+i)
        query = query + q
    query = query[:-10]
    try:
        dataset = pd.read_gbq(query, project_id=project_id, dialect = 'standard')
        df_eu.append(dataset)
    except Exception as e:
        print(e)
df_eu = pd.concat(df_eu)

In [ ]:
df = df_au

In [ ]:
df.creation_time = [datetime.datetime.fromtimestamp(s/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f') for s in df.creation_time]
df.last_modified_time = [datetime.datetime.fromtimestamp(s/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f') for s in df.last_modified_time]

In [ ]:
df.creation_time = [datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S.%f') for i in df.creation_time]
df.last_modified_time = [datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S.%f') for i in df.last_modified_time]
df['gigabytes'] = df.size_bytes/1000000000
df['megabytes'] = df.size_bytes/1000000

In [ ]:
df.to_gbq('BIGQUERY_STATS.datasets_tables_statistics', project_id,if_exists = 'replace')

In [ ]:
df = df.groupby(['dataset_id']).agg({'last_modified_time':max,'gigabytes':sum,'megabytes':sum}).reset_index()
df.to_gbq('BIGQUERY_STATS.datasets_aggregated_statistics', project_id, if_exists = 'replace')